<a href="https://colab.research.google.com/github/swisscheese38/tfcatflap-training/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learn and export the model
According to https://www.tensorflow.org/lite/models/modify/model_maker/object_detection
## Install dependencies

In [2]:
!python --version
!pip --version
!pip install -q tflite-model-maker

Python 3.8.10
pip 22.0.4 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)


## Assert presence of pictures and bounding boxes

Training data should be uploaded in the Jupyter notebook's workspace. There should be a `train`, a `validate` and a `test` folder. Each of them should contain images in `jpeg` format and bounding boxes in `Pascal VOC XML` format. You can use https://www.makesense.ai to create the bounding boxes for your labels in your images.

In [6]:
!pwd
!ls -R

/content
.:
test  train  validate

./test:
img_1674412177951.jpg  img_1674761013859.jpg  img_1675192410433.jpg
img_1674412177951.xml  img_1674761013859.xml  img_1675192410433.xml
img_1674497552657.jpg  img_1674808236542.jpg  img_1675258662302.jpg
img_1674497552657.xml  img_1674808236542.xml  img_1675258662302.xml
img_1674504120619.jpg  img_1674809298789.jpg  img_1675319610246.jpg
img_1674504120619.xml  img_1674809298789.xml  img_1675319610246.xml
img_1674627210835.jpg  img_1674889996204.jpg  img_1675319611278.jpg
img_1674627210835.xml  img_1674889996204.xml  img_1675319611278.xml
img_1674627216842.jpg  img_1674889997238.jpg  img_1675319612310.jpg
img_1674627216842.xml  img_1674889997238.xml  img_1675319612310.xml
img_1674627217874.jpg  img_1674889998269.jpg  img_1675325176998.jpg
img_1674627217874.xml  img_1674889998269.xml  img_1675325176998.xml
img_1674717366096.jpg  img_1674934837268.jpg  img_1675357367517.jpg
img_1674717366096.xml  img_1674934837268.xml  img_1675357367517.xml
img_1


## Load data


In [7]:
from __future__ import annotations
from tflite_model_maker import object_detector
from tflite_model_maker import model_spec

spec = model_spec.get('efficientdet_lite0')
train_data = object_detector.DataLoader.from_pascal_voc(images_dir='train', annotations_dir='train', label_map=['Snout'])
validation_data = object_detector.DataLoader.from_pascal_voc(images_dir='validate', annotations_dir='validate', label_map=['Snout'])
test_data = object_detector.DataLoader.from_pascal_voc(images_dir='test', annotations_dir='test', label_map=['Snout'])

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


## Train model

In [8]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
  6/110 [>.............................] - ETA: 17s - det_loss: 1.7205 - cls_loss: 1.1139 - box_loss: 0.0121 - reg_l2_loss: 0.0630 - loss: 1.7835 - learning_rate: 0.0079 - gradient_norm: 2.9390

110/110 [==============================] - 73s 245ms/step - det_loss: 1.0339 - cls_loss: 0.5844 - box_loss: 0.0090 - reg_l2_loss: 0.0631 - loss: 1.0970 - learning_rate: 0.0065 - gradient_norm: 4.7901 - val_det_loss: 1.5772 - val_cls_loss: 1.0773 - val_box_loss: 0.0100 - val_reg_l2_loss: 0.0632 - val_loss: 1.6404
Epoch 2/50
110/110 [==============================] - 24s 214ms/step - det_loss: 0.5770 - cls_loss: 0.2961 - box_loss: 0.0056 - reg_l2_loss: 0.0633 - loss: 0.6403 - learning_rate: 0.0050 - gradient_norm: 4.9724 - val_det_loss: 1.2294 - val_cls_loss: 0.6548 - val_box_loss: 0.0115 - val_reg_l2_loss: 0.0634 - val_loss: 1.2928
Epoch 3/50
110/110 [==============================] - 22s 204ms/step - det_loss: 0.4962 - cls_loss: 0.2678 - box_loss: 0.0046 - reg_l2_loss: 0.0634 - loss: 0.5597 - learning_rate: 0.0050 - gradient_norm: 5.2285 - val_det_loss: 0.5269 - val_cls_loss: 0.3005 - val_box_loss: 0.0045 - val_reg_l2_loss: 0.0635 - val_loss: 0.5904
Epoch 4/50
110/110 [================

## Test the model

In [9]:
model.evaluate(test_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.60986143,
 'AP50': 1.0,
 'AP75': 0.68423057,
 'APs': -1.0,
 'APm': 0.50555557,
 'APl': 0.60719687,
 'ARmax1': 0.674,
 'ARmax10': 0.706,
 'ARmax100': 0.71,
 'ARs': -1.0,
 'ARm': 0.6,
 'ARl': 0.71224487,
 'AP_/Snout': 0.60986143}

## Export to tensorflow lite model

In [10]:
model.export(export_dir='.')

## Evaluate the tensorflow lite model

In [11]:
model.evaluate_tflite('model.tflite', test_data)


50/50 [==============================] - 142s 3s/step



{'AP': 0.5997981,
 'AP50': 1.0,
 'AP75': 0.65040624,
 'APs': -1.0,
 'APm': 0.5,
 'APl': 0.59692514,
 'ARmax1': 0.666,
 'ARmax10': 0.666,
 'ARmax100': 0.668,
 'ARs': -1.0,
 'ARm': 0.5,
 'ARl': 0.67142856,
 'AP_/Snout': 0.5997981}